In [1]:
import psycopg2
from psycopg2 import sql

In [2]:
def polygon_to_area(polygon_coords):
    conn = psycopg2.connect(host="10.223.72.83" ,port="5433", database = "postgres" , user="postgres" , password="punpuntpasswd")
    try:
        polygon_wkt = "POLYGON((" + ",".join([f"{lon} {lat}" for lon, lat in polygon_coords]) + "))"
        query = f"""
        WITH polygon_geom AS (
            SELECT ST_SetSRID(ST_GeomFromText('{polygon_wkt}'), 4326) AS geom
        )
        SELECT
            SUM(CASE WHEN value = 1 THEN count END) AS count_value1,
            SUM(CASE WHEN value = 2 THEN count END) AS count_value2
        FROM (
            SELECT
                value,
                count
            FROM (
                SELECT
                    (ST_ValueCount(ST_Clip(rast, geom))).*
                FROM
                    raster_results_1,
                    polygon_geom
                WHERE
                    ST_Intersects(rast, geom)
            ) AS counts
        ) AS summary;

        """
        cursor = conn.cursor()
    
        cursor.execute(query, (polygon_wkt,))
    
        result = cursor.fetchone()

        if result:
            rice_area = result[0]*100
            sugarcane_area = result[1]*100
        
        return f"{rice_area, sugarcane_area}"
        
    except Exception as e:
        print(f"Error: {e}")
        
    finally:
        cursor.close()
        conn.close()

In [3]:
polygon_coords = [
    # (102.26539189185458, 16.54356817627857), 
    # (101.95606037474194, 16.08328194532515),
    # (101.93394678768034, 16.647561388673765), 
    # (102.36400115858939, 16.538869127991614), 
    # (102.26539189185458, 16.54356817627857)

    # (101.86092, 15.57712), 
    # (101.87708, 15.57653),
    # (101.87435, 15.56847), 
    # (101.86335, 15.56638), 
    # (101.86092, 15.57712)

    (-93.5164, 42.0347),
    (-93.5164, 42.0481),
    (-93.4910, 42.0481),
    (-93.4910, 42.0347),
    (-93.5164, 42.0347)
]

In [4]:
polygon_to_area(polygon_coords)

0.09671260689932148